In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loder" error.
!pip install distributed

--2022-04-20 12:31:02--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-20 12:31:02--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M   123MB/s    in 0.6s    

2022-04-20 12:31:03 (123 MB/s) - ‘Miniconda3-latest-Linux

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
psi4.core.set_output_file("log.txt")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

mol = psi4.geometry("""
0 1
 C   -1.073986885437   -1.170578359942    0.000002755738
 C    0.953021017198    0.140745143377   -0.000003061297
 C    0.196029366481    1.360797354321   -0.000001202096
 C   -1.155622586106    1.236381947214    0.000002880952
 C   -1.818434996700   -0.031125507024    0.000005467599
 H   -1.510894241728   -2.166796480067    0.000004269828
 H    0.691726656323    2.319296967258   -0.000002742435
 H   -1.770734621411    2.131633416218    0.000004678262
 H   -2.895472126141   -0.076640107909    0.000009377672
 O    2.177846935050   -0.264655641303   -0.000003962413
 H    1.432324292904   -1.412270186206   -0.000002894660
 N    0.288380859812   -1.071403950594   -0.000002249524            
""")

scf_e, wfn = psi4.energy("B3LYP-D3BJ/6-311G(d,p)", return_wfn = True)
print('energy at the {} level of theory:\t{:.4f}'.format("B3LYP-D3BJ/6-311G(d,p)", scf_e))

energy at the B3LYP-D3BJ/6-311G(d,p) level of theory:	-323.5617


In [4]:
psi4.oeprop(wfn, 'WIBERG_LOWDIN_INDICES')
!cat log.txt


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on 447526366ad1
*** at Wed Apr 20 12:48:51 2022

   => Loading Basis Set <=

    Name: 6-311G(D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-5     entry C          line   113 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 
    atoms 6-9, 11 entry H          line    32 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 
    atoms 10      entry O          line   151 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 
    atoms 12      entry N          line   132 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 

   => B3LYP-D3(BJ): Empirical Dispersion <=

    Grimme's -D3 (BJ-damping) Dispersion Correction
    Grimme S.; Ehrlich S.; Goerigk L. (2011), J. Comput. Chem., 32: 1456

        s6 =       1.000000
        s8 =       1.988900
        a1 =       0.398100
        a2 =       4.421100


         ----------------------------

In [5]:
import numpy as np
import pandas as pd
df = pd.DataFrame()

psi4.oeprop(wfn, 'MULLIKEN_CHARGES')

df['B3LYP/6-31G(d)'] = np.array(wfn.atomic_point_charges())
print (df)

    B3LYP/6-31G(d)
0         0.165994
1         0.392642
2        -0.188144
3         0.013313
4        -0.246101
5         0.125442
6         0.116723
7         0.113954
8         0.107335
9        -0.436745
10        0.242167
11       -0.406580


In [6]:
#psi4.oeprop(wfn,'DIPOLE')
dipole_vec = psi4.variable('scf dipole')
dipole = np.sqrt(np.sum(dipole_vec ** 2))
print(f'{dipole:.3f} [D]')

1.398 [D]
